In [1]:
#This code makes the flat galaxy maps for distance calculations.

In [1]:
#import packages and initialize settings
import time
import os
import numpy as np
from astropy.io import fits
import astropy.io.fits as pyfits
from astropy.wcs import WCS
from matplotlib import pyplot as plt
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table

plt.rc('text',usetex=True)
fontsize=20
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

import sys
sys.path.append('/Users/nessmayker/Desktop/PythonFunctions')
from deprojectGalaxy import deproject

In [2]:
# Unpack master table to get SNe information
SNe,SNtype,SNra,SNdec,SNGCRRadArcSec,SNGCRPosAng,Galaxy,GalDist,GalRa,GalDec,GalPosAng,GalIncl,NumSNe,TypesSNe,NatFile,NatBeamSize,NatRes,NatInt,NatErr,NatEW,SNNat,pc60Int,pc60Err,pc60EW,SN60,pc90Int,pc90Err,pc90EW,SN90,pc120Int,pc120Err,pc120EW,SN120,pc150Int,pc150Err,pc150EW,SN150,pc500Int,pc500Err,pc500EW,SN500,pc750Int,pc750Err,pc750EW,SN750,pc1000Int,pc1000Err,pc1000EW,SN1000,pc1250Int,pc1250Err,pc1250EW,SN1250,alphaCO = np.loadtxt('../Data/SNeCO_Master_Table.csv', dtype = str, delimiter = ',',unpack=True)

#generate lists of file names
GalImage, GalErr, GalAlphaCO = [],[],[]
intType = "_co21_broad_mom0"
errType = "_co21_broad_emom0"
preamble = "../ALMAmaps/"
res = '_150pc'

for i in range(len(Galaxy)):
    
    GalName = Galaxy[i]
    alphaGal = GalName.upper()

    if GalName == 'circinus' or GalName == 'ngc0253':
        prefix = "7m+tp"
    elif GalName == 'ngc2775':
        prefix = "12m+7m"
    else:
        prefix = "12m+7m+tp"
                    
    imgStr = preamble + prefix + intType + "/" + GalName + "_" + prefix + intType + res + ".fits"
    errStr = preamble + prefix + errType + "/" + GalName + "_" + prefix + errType + res + ".fits"
    
    if GalName == 'circinus':        
        acoStr = "../alphaCOmaps/ESO097-013_alphaCO21_Sun+20.fits"
    #missing ngc2775 map    
    elif GalName == 'ngc2775':
        acoStr = 'no file'
    else:
        acoStr = "../alphaCOmaps/" + alphaGal + "_alphaCO21_Sun+20.fits"
    
    GalImage.append(imgStr)
    GalErr.append(errStr)
    GalAlphaCO.append(acoStr)

    
GalRas = [float(x) for x in GalRa]
GalDecs = [float(y) for y in GalDec]
centerCoords = [(x,y) for x in GalRas for y in GalDecs]

GalDist = [float(x) for x in GalDist]
GalPosAng = [float(x) for x in GalPosAng]
GalIncl = [float(x) for x in GalIncl]

In [4]:
def distance(x1, x2, y1, y2, galDist):
    #calculate distance between two points (in kpc)
    #x1, y1 = xprime and yprime, x2, y2 = SN coords, dist = distance to galaxy (kpc)
    d = np.sqrt((x2-x1)**2 + (y2-y1)**2)
    x = galDist * np.tan(d)
    return(x)
            
N = 100

for i in range(len(Galaxy)):
    start = time.time()
    filestr = Galaxy[i] + '_'
    inten, err, aco, mass, SNR, ra, dec, dx, dy, xVals, yVals = np.loadtxt("../Data/FlatGalaxyMaps/{}FlatData.txt".format(filestr), delimiter = ' ', unpack=True)
    for j in range(100):
        #assign a random pixel for comparison
        numrows = len(xVals)    
        numcols = len(yVals)
        randX = np.random.randint(low=0, high=numrows+1)
        randY = np.random.randint(low=0, high=numcols+1)
        rX = xVals[randX]
        rY = yVals[randY]

        distRand = []
        #flat map thing with all  the distances from one random point
        distRand = distance(xVals, rX, yVals, rY, GalDist[i]) 

        filestr = Galaxy[i] + '_Area_' + str(j) + '_'
        with open ("../Data/RandomMaps/{}FlatData.txt".format(filestr), "w") as text_file:
            print('#Intensity, Error, AlphaCO,   Mass,   SNR,      Ra,      Dec,      dx,      dy,     xVals,     yVals,     randomDist', file = text_file)
            for k in range(len(inten)):
                print(inten[k], err[k], aco[k], mass[k], SNR[k], ra[k], dec[k], dx[k], dy[k], xVals[k], yVals[k], distRand[k], file = text_file)

    end = time.time()
    print("Time to run 100 random placement (area) experiments: ", end-start)

KeyboardInterrupt: 